In [5]:
import attr
from bs4 import BeautifulSoup
import requests
import pandas as pd
import warnings

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# warnings.filterwarnings('ignore')


In [42]:
wget

NameError: name 'wget' is not defined

In [40]:
page = requests.get( url, verify=False, headers={'User-Agent':  user_agent})
soup = BeautifulSoup(page.content, 'html.parser')
pages = soup.find_all('span', class_="page")
pages = soup.find_all("div")
pages

[<div class="container">
 <script>document.getElementsByClassName("container")[0].style.display = "none";</script>
 <noscript>This page requires JavaScript!</noscript>
 <div class="row center-block">
 <img alt="0" class="logo-realtor" src="https://b2cdata.marketing.moveaws.com/forcecaptcha/images/realtor.png"/>
 <br/>
 <br/>
 <h1 class="title-force">Pardon Our Interruption...</h1>
 </div>
 <div class="row">
 <div class="content col-lg-2 col-sm-2"></div>
 <div class="container-force content col-lg-8 col-sm-8">
 <p>
         As you were browsing, something about your browser made us think you might be a bot. There are a few reasons this might happen, including:
       </p>
 <ul>
 <li>You're a power user moving through this website with super-human speed</li>
 <li>You've disabled JavaScript and/or cookies in your web browser</li>
 <li>A third-party browser plugin is preventing JavaScript from running.</li>
 </ul>
 <p>
           Reference ID: #e16d5e17-6861-11ec-a62e-4e6d684b575a
       <

In [16]:
# url = "https://www.realtor.com/soldhomeprices/Woodbury_MN"
url = "https://www.realtor.com/realestateandhomes-search/Chicago_IL/show-recently-sold"
user_agent ='Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2272.101 Safari/537.36'
full_home_list = []
pages = 0

########################################################################################################################

i = 1 # for i in range(2, pages + 1):
page = requests.get(url + "/pg-" + str(i), verify=False, headers={'User-Agent': user_agent})
soup = BeautifulSoup(page.content, 'html.parser')


ipages = int(pages[3].get_text().replace('\n', ''))
pages = ipages
full_home_list = parse_homes(soup)


parse_homes(soup, soup)


# full_home_list = full_home_list + parse_homes(soup)
# print(f"processed {i} of {pages}")
# df = pd.DataFrame(full_home_list)
# df = clean_and_convert(df)
# df.head()



def sanitize(item):
    return item.get_text().replace('\n', '')


def parse_homes(soup):
    details = soup.find_all('div', class_='detail-wrap')
    homes = []
    for house in details:
        home = {}
        home['price'] = sanitize(house.find(class_="data-price"))
        home['type'] = sanitize(house.find(class_="property-type"))
        home['beds'] = sanitize(house.find(attrs={"data-label": "property-meta-beds"}))
        home['baths'] = sanitize(house.find(attrs={"data-label": "property-meta-baths"}))
        home['sqft'] = sanitize(house.find(attrs={"data-label": "property-meta-sqft"}))
        home['garages'] = sanitize(house.find(attrs={"data-label": "property-meta-garage"}))
        home['street'] = sanitize(house.find(class_="listing-street-address"))
        home['city'] = sanitize(house.find(class_="listing-city"))
        home['region'] = sanitize(house.find(class_="listing-region"))
        home['postal'] = sanitize(house.find(class_="listing-postal"))
        homes.append(home)
    return homes



def clean_and_convert(df):
    df['bath_convert'] = df.baths.apply(lambda x: x.split(" ")[0].replace("+", ""))
    df['bed_convert'] = df.beds.apply(lambda x: x.split(" ")[0].replace("+", ""))
    df['garage_convert'] = df.garages.apply(lambda x: x.split(" ")[0].replace("+", ""))
    df['sqft_convert'] = df.sqft.apply(lambda x: x.split(" ")[0].replace("+", "").replace(",", ""))
    df['price_convert'] = df.price.apply(lambda x: x.replace("$", "").replace(",", ""))
    # df.drop(df[df.bath_convert == ""].index, inplace=True, axis=0)
    # df.drop(df[df.bed_convert == ""].index, inplace=True, axis=0)
    df.garage_convert = df.garage_convert.apply(lambda x: 0 if x == '' else x)
    df.drop(df[df.sqft_convert == ''].index, inplace=True, axis=0)
    df.bath_convert = df.bath_convert.astype('float')
    df.bed_convert = df.bed_convert.astype('float')
    df.garage_convert = df.garage_convert.astype('float')
    df.sqft_convert = df.sqft_convert.astype('float')
    df.price_convert = df.price_convert.astype('float')
    return df

        


TypeError: parse_homes() takes 1 positional argument but 2 were given